In [42]:
import pandas as pd
import numpy as np
from datetime import datetime 
start_time = datetime.now() 

marg_port = pd.read_excel('marg_port3.xlsx')
marg_port['Date'] = pd.to_datetime(marg_port['Date'])
marg_port.set_index('Date', inplace=True)
marg_port.dropna(subset=['ID'], inplace=True)

perc = [1, 0.5, 0.5]
long_risk_rate = [1, 1.2, 1.3]
short_risk_rate = [1, 1.2, 1.2]

def bar(df, day, time):
    while day <= time:
        print(f'День {day}')
        display('0 шаг - входные данные', df)
        
        #Фильтры на активы
        money_filter = df['Class_equity'] == 'Деньги' 
        stocks_filter = df['Class_equity'] == 'Акции'
        eur_bonds_filter = df['Class_equity'] == 'Еврооблигации'
        futures_filter = df['Class_equity'] == 'Фьючерс'
        currency_filter = df['Class_equity']  == 'Валюта'
        
        #Создаем таблицы
        money_table = df.where(money_filter).dropna(how='all')
        currency_table = df.where(currency_filter).dropna(how='all')
        stocks_table = df.where(stocks_filter).dropna(how='all')
        eur_bonds_table = df.where(eur_bonds_filter).dropna(how='all')
        futures_table = df.where(futures_filter).dropna(how='all')
        
        #Расчеты для денег
        
        #Расчеты для валюты
        currency_table['Price_FX'] = currency_table['Price_FX'].apply(lambda x: x * perc[day]).round(2)
        
        #Расчеты для акций
        stocks_table['Prices'] = stocks_table['Prices'].apply(lambda x: x * perc[day]).round(2)
        
        #Расчеты для еврооблигаций
        eur_bonds_table['Prices'] = eur_bonds_table['Prices'].apply(lambda x: x * perc[day]).round(2)
        eur_bonds_table['Price_FX'] = eur_bonds_table['Price_FX'].apply(lambda x: x * perc[day]).round(2)
        
        main_table = pd.concat([money_table, currency_table, stocks_table, eur_bonds_table, futures_table], ignore_index=False, sort=False)
        
        #Прочие расчеты
        main_table['Short_rate'] = main_table['Short_rate'].apply(lambda x: x * short_risk_rate[day]).round(2)
        main_table['Long_rate'] = main_table['Long_rate'].apply(lambda x: x * long_risk_rate[day]).round(2)
        main_table['Position'] = (main_table['Quantity'] * main_table['Prices'] * main_table['Liquidty_factor'] * main_table['Price_FX']).round(2)
        main_table['First_margin'] = main_table['Position'].apply(lambda x: -min(-long_risk_rate[day] * x, short_risk_rate[day] * x)).round(2)
        main_table['First_margin'] *= main_table['Short_rate'].round(2)
        main_table['Last_margin'] =  (main_table['First_margin']*0.5).round(2)
        main_table['NPR1'] =  (main_table['Position'] -  main_table['First_margin']).round(2)
        main_table['NPR2'] = (main_table['Position'] - main_table['Last_margin']).round(2)
        display('1 шаг - расчет показателей Position, First_margin, Last_margin, NPR1, NPR2', main_table)
        
        #Группировка по портфелям
        group = main_table.groupby('ID_Portfolio').sum()
        display('2 шаг - группировка по портфелям',group)
        
        #Собираем только неотрицательные портфели
        if day == time:
            new_group_portf = group.where(0 < group['NPR2']).dropna(how='all')
            print(f'День{day}, НПР{day}')
        else:
            new_group_portf = group.where(0 < group['NPR1']).dropna(how='all')
            print(f'День{day}, НПР{day}')
        new_group_portf.reset_index(inplace=True)
        display('3 шаг - в n+1 день перейдут только следующие портфели', new_group_portf)
        
        #Проверяем, какие портфели остались
        plus_portf = [i for i in new_group_portf['ID_Portfolio']]
        if len(plus_portf) == 0:
            print(f'Дефолт всех портфелей на дату t{day}')
            return
        
        #По ID_Portfolio собираем позиции в оставшихся портфелях в новый датафрейм
        cols = list(main_table.columns)
        data = []
        for row in main_table.itertuples():
            row_dict = {k:v for k,v in zip(cols, row[1:])}
            #print('row_dict',row_dict)
            #print(row_dict['group'])
            if row_dict['ID_Portfolio'] in plus_portf:
                data.append(row_dict)
        #print(data)
        new_set = pd.DataFrame.from_records(data)
        if day == time:
            display('Расчет завершился, оставшиеся портфели:', new_set)
            global final_dataset
            final_dataset = new_set
            return final_dataset
        else:
            display('4 шаг - оставляем позиции в неотрицательных портфелях, данные позиции перейдут на следующий день', new_set)
        #new_set.set_index('Index')
        bar(new_set, day + 1, time)
        return new_set
day = 1
time = 2
bar(marg_port, day, time)

#Сводная таблица по портфелям
display('Сводная таблица', final_dataset[['ID_Portfolio','Position', 'First_margin', 'Last_margin', 'NPR1','NPR2']].groupby(['ID_Portfolio']).sum())

time_elapsed = datetime.now() - start_time
print('Время расчета (hh:mm:ss.ms) {}'.format(time_elapsed))

День 1


'0 шаг - входные данные'

,RISK_Group,ID_Portfolio,ID_Name,Class_equity,ID,FX,Quantity,Prices,Price_FX,Variation_marg,Liquidty_factor,Short_rate,Long_rate
Date,,,,,,,,,,,,,
2020-12-30,КСУР,ФЛ111,Лукойл,Акции,LKOH,RUB,100.0,5717.0,1.0,0.0,1.0,0.2100,0.2100
2020-12-30,КСУР,ФЛ111,Сбер,Акции,SBER,RUB,-300.0,268.0,1.0,0.0,1.0,0.2100,0.2100
2020-12-30,КСУР,ФЛ111,Абрау,Акции,ABRD,RUB,1000.0,199.5,1.0,0.0,0.0,1.0000,1.0000
2020-12-30,КСУР,ФЛ111,Евробонд,Еврооблигации,XS2252958752,USD,1.0,995.0,75.0,0.0,1.0,0.2200,0.2200
2020-12-30,КСУР,ФЛ111,Лукойл,Фьючерс,LKOH-3.21,RUB,2.0,57950.0,1.0,-500.0,0.0,0.1021,0.1021
2020-12-30,КСУР,ФЛ112,Рубли,Акции,RUB,RUB,50000.0,1.0,1.0,0.0,1.0,0.0000,0.0000
2020-12-30,КСУР,ФЛ112,Доллары,Валюта,USD,USD,3000.0,1.0,75.0,0.0,1.0,0.1000,0.1000
2020-12-30,КСУР,ФЛ112,Лукойл,Акции,LKOH,RUB,200.0,5717.0,1.0,0.0,1.0,0.2100,0.2100
2020-12-30,КСУР,ФЛ112,Сбер,Акции,SBER,RUB,500.0,268.0,1.0,0.0,1.0,0.2100,0.2100


'1 шаг - расчет показателей Position, First_margin, Last_margin, NPR1, NPR2'

,RISK_Group,ID_Portfolio,ID_Name,Class_equity,ID,FX,Quantity,Prices,Price_FX,Variation_marg,Liquidty_factor,Short_rate,Long_rate,Position,First_margin,Last_margin,NPR1,NPR2
Date,,,,,,,,,,,,,,,,,,
2020-12-30,КСУР,ФЛ113,Рубли,Деньги,RUB,RUB,400000.0,1.00,1.0,0.0,1.0,0.00,0.00,400000.00,0.00,0.00,400000.0,400000.00
2020-12-30,КСУР,ФЛ112,Доллары,Валюта,USD,USD,3000.0,1.00,37.5,0.0,1.0,0.12,0.12,112500.00,16200.00,8100.00,96300.0,104400.00
2020-12-30,КСУР,ФЛ111,Лукойл,Акции,LKOH,RUB,100.0,2858.50,1.0,0.0,1.0,0.25,0.25,285850.00,85755.00,42877.50,200095.0,242972.50
2020-12-30,КСУР,ФЛ111,Сбер,Акции,SBER,RUB,-300.0,134.00,1.0,0.0,1.0,0.25,0.25,-40200.00,12060.00,6030.00,-52260.0,-46230.00
2020-12-30,КСУР,ФЛ111,Абрау,Акции,ABRD,RUB,1000.0,99.75,1.0,0.0,0.0,1.20,1.20,0.00,0.00,0.00,0.0,0.00
2020-12-30,КСУР,ФЛ112,Рубли,Акции,RUB,RUB,50000.0,0.50,1.0,0.0,1.0,0.00,0.00,25000.00,0.00,0.00,25000.0,25000.00
2020-12-30,КСУР,ФЛ112,Лукойл,Акции,LKOH,RUB,200.0,2858.50,1.0,0.0,1.0,0.25,0.25,571700.00,171510.00,85755.00,400190.0,485945.00
2020-12-30,КСУР,ФЛ112,Сбер,Акции,SBER,RUB,500.0,134.00,1.0,0.0,1.0,0.25,0.25,67000.00,20100.00,10050.00,46900.0,56950.00
2020-12-30,КСУР,ФЛ111,Евробонд,Еврооблигации,XS2252958752,USD,1.0,497.50,37.5,0.0,1.0,0.26,0.26,18656.25,5820.75,2910.38,12835.5,15745.87


...

'Расчет завершился, оставшиеся портфели:'

,RISK_Group,ID_Portfolio,ID_Name,Class_equity,ID,FX,Quantity,Prices,Price_FX,Variation_marg,Liquidty_factor,Short_rate,Long_rate,Position,First_margin,Last_margin,NPR1,NPR2
0,КСУР,ФЛ113,Рубли,Деньги,RUB,RUB,400000.0,1.00,1.00,0.0,1.0,0.00,0.00,400000.00,0.0000,0.00,400000.00,400000.00
1,КСУР,ФЛ112,Доллары,Валюта,USD,USD,3000.0,1.00,18.75,0.0,1.0,0.14,0.16,56250.00,10237.5000,5118.75,46012.50,51131.25
2,КСУР,ФЛ111,Лукойл,Акции,LKOH,RUB,100.0,1429.25,1.00,0.0,1.0,0.30,0.32,142925.00,55740.7500,27870.38,87184.25,115054.62
3,КСУР,ФЛ111,Сбер,Акции,SBER,RUB,-300.0,67.00,1.00,0.0,1.0,0.30,0.32,-20100.00,7236.0000,3618.00,-27336.00,-23718.00
4,КСУР,ФЛ111,Абрау,Акции,ABRD,RUB,1000.0,49.88,1.00,0.0,0.0,1.44,1.56,0.00,0.0000,0.00,0.00,0.00
5,КСУР,ФЛ112,Рубли,Акции,RUB,RUB,50000.0,0.25,1.00,0.0,1.0,0.00,0.00,12500.00,0.0000,0.00,12500.00,12500.00
6,КСУР,ФЛ112,Лукойл,Акции,LKOH,RUB,200.0,1429.25,1.00,0.0,1.0,0.30,0.32,285850.00,111481.5000,55740.75,174368.50,230109.25
7,КСУР,ФЛ112,Сбер,Акции,SBER,RUB,500.0,67.00,1.00,0.0,1.0,0.30,0.32,33500.00,13065.0000,6532.50,20435.00,26967.50
8,КСУР,ФЛ111,Евробонд,Еврооблигации,XS2252958752,USD,1.0,248.75,18.75,0.0,1.0,0.31,0.34,4664.06,1879.6168,939.81,2784.44,3724.25
9,КСУР,ФЛ111,Лукойл,Фьючерс,LKOH-3.21,RUB,2.0,57950.00,1.00,-500.0,0.0,0.14,0.16,0.00,0.0000,0.00,0.00,0.00


'Сводная таблица'

,Position,First_margin,Last_margin,NPR1,NPR2
ID_Portfolio,,,,,
ФЛ111,127489.06,64856.3668,32428.19,62632.69,95060.87
ФЛ112,388100.00,134784.0000,67392.00,253316.00,320708.00
ФЛ113,400000.00,0.0000,0.00,400000.00,400000.00


Время расчета (hh:mm:ss.ms) 0:00:00.382765
